In [7]:
import awkward as ak
import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection
import json
import numpy as np
import mplhep as hep
import matplotlib.pyplot as plt

In [8]:
filename = "/home/prayag/Coffea/data/3BF5BC57-5D24-1C4D-B277-7B0BAA5DF3C7.root"
events = NanoEventsFactory.from_root(
    filename,
    schemaclass=NanoAODSchema.v7
).events()

In [1]:
from coffea import util

In [3]:
util.load("Zjetsnunu_ZJets_NuNu_from_1_to_20.coffea")

{'ZJets_NuNu': {'Z1Jets_NuNu_ZpT_50To150_18': {'Cutflow': {'Total events': 5315107,
    'MET > 200 GeV': 4370,
    'no electrons': 4370,
    'no muons': 4307,
    'no photons': 4276,
    'no taus': 4276,
    'jet pt > 30': 183,
    'jet abs(eta) < 2.5': 152,
    'deltaphi(jets, MET) > 0.4': 152,
    'At least two bjets': 0,
    'bjet1 pt > 50 GeV and bjet2 pt > 30': 0,
    'Additional Jets <= 2': 0,
    'dijet mass between 100 Gev to 150 GeV': 0,
    'dijet pt > 100 GeV': 0},
   'Histograms': {'met_pt_hist': Hist(Regular(50, 0, 1000, label='Axis 0'), storage=Double()),
    'met_phi_hist': Hist(Regular(30, -3.14, 3.14, label='Axis 0'), storage=Double()),
    'leadingjets_pt_hist': Hist(Regular(25, 0, 500, label='Axis 0'), storage=Double()),
    'leadingjets_eta_hist': Hist(Regular(30, -3, 3, label='Axis 0'), storage=Double()),
    'leadingjets_phi_hist': Hist(Regular(30, -3.14, 3.14, label='Axis 0'), storage=Double()),
    'leadingjets_mass_hist': Hist(Regular(25, 0, 500, label='Axis 0'

In [4]:
util.load("Zjetsnunu_MET_Run2018_from_1_to_20.coffea")

{'MET_Run2018': {'MET_Run2018A': {'Cutflow': {'Total events': 22985157,
    'lumimask': 20020999,
    'MET trigger': 8855754,
    'MET filters': 8143238,
    'MET > 200 GeV': 916265,
    'no electrons': 916265,
    'no muons': 851722,
    'no photons': 805851,
    'no taus': 805851,
    'jet pt > 30': 39327,
    'jet abs(eta) < 2.5': 20328,
    'deltaphi(jets, MET) > 0.4': 20328,
    'At least two bjets': 376,
    'bjet1 pt > 50 GeV and bjet2 pt > 30': 376,
    'Additional Jets <= 2': 208,
    'dijet mass between 100 Gev to 150 GeV': 24,
    'dijet pt > 100 GeV': 24},
   'Histograms': {'met_pt_hist': Hist(Regular(50, 0, 1000, label='Axis 0'), storage=Double()) # Sum: 208.0,
    'met_phi_hist': Hist(Regular(30, -3.14, 3.14, label='Axis 0'), storage=Double()) # Sum: 208.0,
    'leadingjets_pt_hist': Hist(Regular(25, 0, 500, label='Axis 0'), storage=Double()) # Sum: 178.0 (208.0 with flow),
    'leadingjets_eta_hist': Hist(Regular(30, -3, 3, label='Axis 0'), storage=Double()) # Sum: 208.0